In [3]:
n_docs

(500, 500)

In [7]:
import psutil
import os
reserved_memory = (1024**3)//2
available_memory = psutil.virtual_memory()[0]
usable_memory = (available_memory - reserved_memory)//1024**2

my_env = os.environ.copy()
my_env["TERRIER_HEAP_MEM"] = f"{usable_memory}M"

In [4]:
import subprocess
from tqdm import tqdm
import itertools

terrier_path = "/ssd2/arthur/terrier-core"
index_path = "/ssd2/arthur/TREC2019/data/index/terrier/msmarco_docs"
data_path = "/ssd2/arthur/TREC2019/data"
cmd = """{0}/bin/terrier batchretrieve
        tb-I {1}
        -o {0}/var/results/run.msmarco_docs.bm25_qe_{2}docs_{3}terms.res
        -t {2}/dev.topics -w BM25 -q
        -Dtrec.querying.dump.settings=false -Dtrec.output.format.length=1000 -c c:0.87
        -Dexpansion.terms={3} -Dexpansion.documents={2}"""
n_docs_range = [5, 10, 15, 20, 50, 100, 150, 200, 250, 300, 350, 500]
n_terms_range = [5, 10, 15, 20, 50, 100, 150, 200, 250, 300, 350, 500]

for n_docs, n_terms in tqdm(itertools.product(n_docs_range, n_terms_range)):
    cmd_to_run = cmd.format(terrier_path, index_path, data_path, n_docs, n_terms)
    subprocess.run(cmd_to_run.split())
    print(cmd_to_run, env=my_env)

0it [00:00, ?it/s]

5 5


In [8]:
cmd_to_run

'/ssd2/arthur/terrier-core/bin/terrier batchretrieve\n        -I /ssd2/arthur/TREC2019/data/index/terrier/msmarco_docs\n        -o /ssd2/arthur/terrier-core/var/results/run.msmarco_docs.bm25_qe_/ssd2/arthur/TREC2019/datadocs_(500, 500)terms.res\n        -t /ssd2/arthur/TREC2019/data/dev.topics -w BM25 -q\n        -Dtrec.querying.dump.settings=false -Dtrec.output.format.length=1000 -c c:0.87\n        -Dexpansion.terms=(500, 500) -Dexpansion.documents=/ssd2/arthur/TREC2019/data'

In [ ]:
cmd = "/ssd2/arthur/terrier-core/bin/terrier batchevaluate -f -q {}".format(os.path.join(data_dir, "msmarco-docdev-qrels.tsv"))
output = subprocess.run(cmd.split(), capture_output=True)
print(output.stdout.decode("utf-8").split("\n"))


In [14]:
ANSERINI="/tudelft.net/staff-umbrella/punchbag/msmarco/data/anserini.jar"
INDEX="/tudelft.net/staff-umbrella/punchbag/msmarco/data/lucene-index.msmarco-doc.pos+docvectors+rawdocs"
OUTPUT="/tudelft.net/staff-umbrella/punchbag/msmarco/data/results"
TOPICS="/tudelft.net/staff-umbrella/punchbag/msmarco/data/results/msmarco-docdev-qrels.tsv"
PARAMS="--partition=general --qos=short --time=4:00:00 --cpus-per-task=2 --mem-per-cpu=4096 --mail-type=END"

for i in [5, 10, 15, 20, 50, 100, 150, 200, 250, 300, 350, 500]:
    for j in [5, 10, 15, 20, 50, 100, 150, 200, 250, 300, 350, 500]:    
        cmd=f"srun {PARAMS} java -cp $ANSERINI io.anserini.search.SearchCollection -topicreader Tsv -index $INDEX -topics $TOPICS -output $OUTPUT/bm25+rm3_{i}_{j}.run -bm25 -rm3 -rm3.fbDocs={i} -rm3.fbTerms={j} &"
        print(cmd)

srun --partition=general --qos=short --time=4:00:00 --cpus-per-task=2 --mem-per-cpu=4096 --mail-type=END java -cp $ANSERINI io.anserini.search.SearchCollection -topicreader Tsv -index $INDEX -topics $TOPICS -output $OUTPUT/bm25+rm3_10_5.run -bm25 -rm3 -rm3.fbDocs=10 -rm3.fbTerms=5 &
srun --partition=general --qos=short --time=4:00:00 --cpus-per-task=2 --mem-per-cpu=4096 --mail-type=END java -cp $ANSERINI io.anserini.search.SearchCollection -topicreader Tsv -index $INDEX -topics $TOPICS -output $OUTPUT/bm25+rm3_10_10.run -bm25 -rm3 -rm3.fbDocs=10 -rm3.fbTerms=10 &
srun --partition=general --qos=short --time=4:00:00 --cpus-per-task=2 --mem-per-cpu=4096 --mail-type=END java -cp $ANSERINI io.anserini.search.SearchCollection -topicreader Tsv -index $INDEX -topics $TOPICS -output $OUTPUT/bm25+rm3_10_15.run -bm25 -rm3 -rm3.fbDocs=10 -rm3.fbTerms=15 &
srun --partition=general --qos=short --time=4:00:00 --cpus-per-task=2 --mem-per-cpu=4096 --mail-type=END java -cp $ANSERINI io.anserini.search.S